# Quickstart

## Using the File API Client

Using the notebook utilities provided by `evo-sdk-common` you can easily interact with the File API in a Jupyter notebook environment.

In [ ]:
from evo.notebooks import ServiceManagerWidget

manager = await ServiceManagerWidget.with_auth_code(
    client_id="your-client-id",
    cache_location="./notebook-data",
).login()

## FileAPIClient

The `FileAPIClient` wraps endpoint functionality to provide a cohesive interface to the underlying API client implementation.

In [ ]:
from evo.files import FileAPIClient

environment = manager.get_environment()
connector = manager.get_connector()

file_client = FileAPIClient(environment, connector)
service_health = await file_client.get_service_health()

print(f"File API is {service_health.status.name.lower()}")

### List files

Listing content is simple, just call the `FileAPIClient.list_files()` method.

In [ ]:
offset = 0
while True:
    page = await file_client.list_files(offset=offset, limit=10)
    if offset == 0:
        print(f"Found {page.total} file{'' if page.total == 1 else 's'}")
    for file in page:
        print(f"{file.path}: ({file.id}) Version: {file.version_id}")
    if page.is_last:
        break
    offset = page.next_offset

In [ ]:
from evo.files import FileMetadata

service_files = await file_client.list_all_files(limit_per_request=50)


# Pretty print all files.
def display_files(files: list[FileMetadata]):
    n_files = len(files)
    print(f"Found {n_files} file{'' if n_files == 1 else 's'}")
    for file in files:
        print(f"{file.path}: ({file.id}) Version: {file.version_id}")


display_files(service_files)

### Upload/update a file by path

In [ ]:
path = "points.csv"

# Prepare an upload context by path.
ctx = await file_client.prepare_upload_by_path(path)

# Use the upload context to upload the file.
await ctx.upload_from_path("data/points.csv", connector.transport)

### Get a file by path

In [ ]:
# Get the FileMetadata
points = await file_client.get_file_by_path(path)
print(f"Found {points.path}: UUID({points.id}) Version:{points.version_id}")

### Get the versions of the file by path

In [ ]:
versions = await file_client.list_versions_by_path(path)
print(f"Versions found for {path}:")
for version in versions:
    print(version.version_id)

### Download a specific version by path and version ID

In [ ]:
# Get a specific version by path and version ID. Change the list index to pick a different version.
version_id = versions[0].version_id
print(f"Fetching version {version_id}")
points_version = await file_client.get_file_by_path(path, version_id=version_id)
print(f"Found {points_version.path}: UUID({points_version.id}) Version:{points_version.version_id}")

print("---")

# Download the version
print(f"Downloading version {version_id}")

# Prepare a download by path and version ID.
ctx = await file_client.prepare_download_by_path(path, version_id=version_id)

# Use the download context to download the file.
file_location = await ctx.download_to_cache(manager.cache, connector.transport)
print(f"File downloaded to {str(file_location)}")

### Delete a file by path

In [ ]:
from evo.common.exceptions import NotFoundException

await file_client.delete_file_by_path(path)
try:
    points = await file_client.get_file_by_path(path)
    print("File was not deleted!")
except NotFoundException:
    print("File deleted successfully")

### Find ID of a file

In [ ]:
# List items and copy one of the file IDs
file_id = service_files[0].id
print(file_id)

### Get a file by ID

In [ ]:
# Get points and versions by ID
points = await file_client.get_file_by_id(file_id)
print(f"Found {points.path}: ({points.id}) Version: {points.version_id}")

### Get the versions of the file

In [ ]:
versions = await file_client.list_versions_by_id(file_id)
print(f"Versions found for {file_id}:")
for version in versions:
    print(version.version_id)

### Download a specific version by ID and version ID

In [ ]:
# Get a specific version by ID and version ID
version_id = versions[0].version_id
print(f"Fetching version {version_id}")
points_version = await file_client.get_file_by_id(file_id, version_id=version_id)
print(f"Found {points_version.path}: UUID({points_version.id}) Version:{points_version.version_id}")

print("---")

# Download version
print(f"Downloading version {version_id}")

# Prepare a download context with the file ID and version ID
ctx = await file_client.prepare_download_by_id(file_id, version_id=version_id)

# Use the download context to download the file
file_location = await ctx.download_to_cache(manager.cache, connector.transport)
print(f"File downloaded to {str(file_location)}")

### Update a file by ID

In [ ]:
# Prepare an upload context using the file ID
# The file ID must already exist in the service
ctx = await file_client.prepare_upload_by_id(file_id)

# Use the upload context to upload the file
await ctx.upload_from_path("data/points_2.csv", connector.transport)

### Delete a file by ID

In [ ]:
from evo.common.exceptions import DefaultTypedError

await file_client.delete_file_by_id(file_id)
try:
    points = await file_client.get_file_by_id(file_id)
    print("File was not deleted!")
except DefaultTypedError as e:
    if e.status == 410:
        print("File deleted successfully")
    else:
        print("Unexpected error occurred")

## API documentation

For more information about the File API, visit the [Seequent developer portal](https://developer.seequent.com/docs/guides/file).